## Utilizando GANs com imagens

Apesar de já termos utilizado GANs com dados mais simples, aplicar esses modelos em dados mais complexos tem seus próprios desafios.

Nesta aula treinaremos um modelo GAN na base de dados MNIST, tentando fazer com que o gerador crie amostras sintéticas de imagens de dígitos.

O primeiro passo é carregar a base de treinamento, que faremos através do [tensorflor_datasets](https://www.tensorflow.org/datasets?hl=pt-br)

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='train', 
    batch_size=-1, 
    as_supervised=True,
))
train_image = image
train_labels = label
image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test', 
    batch_size=-1, 
    as_supervised=True,
))

test_image = image
test_labels = label

A base de treinamento contém imagens (28x28x1 - ou seja, preto e branco) com dígitos escritos e sua respectiva label:

In [ ]:
print(label[:3])
plt.imshow(image[0].reshape(28,28),cmap="gray"), plt.figure()
plt.imshow(image[1].reshape(28,28),cmap="gray"), plt.figure()
plt.imshow(image[2].reshape(28,28),cmap="gray")

A label não interessa para nossa aplicação de geração automatizada, portanto preparamos nossa base de treinamento e teste apenas com as imagens. Também serão normalizadas as imagens para que tenham valores entre -1 e 1.

In [ ]:

#Implemente uma rotina de normalização em que train_dataset seja normalizado entre -1 e 1 
# (dica: imagens preto e branco não normalizadas vão de 0 a 255)
train_dataset = #Seu Codigo normalizando train_image
test_dataset = #Seu Codigo normalizando test_image
train_dataset = train_dataset.reshape((len(train_dataset),28,28,1))
test_dataset = test_dataset.reshape((len(test_dataset),28,28,1))

#Aqui deve ser impresso (60000, 28, 28, 1)
print(train_dataset.shape)

Durante o treinamento vai ser útil visualizar as imagens que estão sendo geradas pela rede, portanto precisamos de uma função para facilitar a visualização das imagens

In [ ]:
def print_images(samples):
    plt.close()
    for i in range(25):
        # define subplot
        plt.subplot(5, 5, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(samples[i].reshape((28,28)), cmap='gray_r')
    plt.show()
#As 25 primeiras imagens serão impressas
print_images(train_dataset[:25])

Podemos então construir nossa GAN conforme aprendemos na última aula. Dado que estamos lidando com imagens, podemos nos aproveitar de arquiteturas especializadas para o processamento de imagens, tais como CNNs. Mas inicialmente, construa o modelo baseando-se primariamente em "fully-connected layers".

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Softmax
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2DTranspose, LeakyReLU, Dropout, Input
from tensorflow.keras.layers import UpSampling2D, Conv2D, BatchNormalization,Reshape, Activation, Dense, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras import initializers
import numpy as np

def gan_generator_model(dim_random):
    model = Sequential()
    #Sua arquitetura para o gerador (a saida deve ser no formato (?, 28, 28, 1))
    return model

def gan_discriminator_model():
    model = Sequential()
    #Sua arquitetura para o discriminador (entrada (?, 28, 28, 1) e saida com 1 dimensao entre 0 e 1)
    return model

def define_gan(generator, discriminator):
    # connect generator and discriminator
    discriminator.trainable = False
    ganInput = Input(shape=(dim_random_vector,))
    x = generator(ganInput)
    ganOutput = discriminator(x)
    gan = Model(inputs=ganInput, outputs=ganOutput)
    opt = Adam(lr=0.0002, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer=opt)
    return gan

dim_random_vector = 100
generator = gan_generator_model(dim_random_vector)
generator.summary()
discriminator = gan_discriminator_model()
opt = # Otimizador para o Discriminador
discriminator.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
discriminator.summary()
gan = define_gan(generator, discriminator)
gan.summary()

Antes de realizar o treinamento, vamos também gerar funções intermediárias para a geração de batches

In [ ]:
def generate_batch_discriminator(generator,batch_size, treino = True):
    half_batch = int(batch_size/2)
    
    batch = np.zeros((batch_size, 28, 28, 1))
    batch_y = np.zeros(batch_size)
    
    #Pega exemplos aleatoriamente da base de treinamento ou teste
    if treino:
        random_indices = np.random.choice(len(train_dataset), size=half_batch, replace=False)
        batch[:half_batch, :, :, :] = train_dataset[random_indices, :, :, ]
    else:
        random_indices = np.random.choice(len(test_dataset), size=half_batch, replace=False)
        batch[:half_batch, :, :, :] = test_dataset[random_indices, :, :, :]
    #Labels das instâncias reais já são 0, entao nao há necessidade de mudar batch_y
    
    # A variável aleatória de entrada do gerador vai seguir uma distribuição normal
    batch[half_batch:, :] = generator.predict(np.random.normal(0, 1, size=[half_batch, dim_random_vector]))
    batch_y[half_batch:] += 1. 
    return batch, batch_y

def generate_batch_gan(batch_size):
    x_gan = np.random.normal(0, 1, size=[batch_size, dim_random_vector])
    #Neste caso criamos labels invertidos para os exemplos falsos, porque a funcao de custo
    # do gerador é o inverso do discriminador
    y_gan = np.zeros((batch_size)) 
    return x_gan,y_gan

Agora podemos realizar o treinamento da GAN

In [ ]:

def train_gan(generator,discriminator,gan):
    batch_size = 128
    epochs = 20000

    test_size = 100
    half_test = int(test_size /2)

    #Erros do discriminador e gerador, e percentual de acerto no teste
    errors_discrim = np.zeros((epochs))
    errors_generator = np.zeros((epochs))
    perc_discrim = np.zeros((epochs))

    #Utilizado para a visualização de imagens não travar o treinamento
    plt.ion()

    for i in range(epochs):
        #Implementar o Treinamento da GAN conforme visto na aula 1. Primeiro treinar discriminador, 
        # depois treinar o gerador através da variável "gan". Para isso utilizar aa funcao generate_batch_discriminator
        # -----------------
        # Seu Codigo
        # ---------------------
        #periodicamente
        if i%1000 == 0:
            print("epoch: "+str(i))
            print_images(generator.predict(np.random.normal(0, 1, size=[50, dim_random_vector])))
            print("Errors Discrimin: " + str(errors_discrim[i]))
            print("Errors Generator:" + str(errors_generator[i]))
            print("Perc Discrim:" + str(perc_discrim[i]))
train_gan(generator, discriminator, gan)

Podemos avaliar a qualidade das instâncias geradas em comparação com as reais, também avaliar a progressão de erros para o gerador e discriminador

In [ ]:
print_images(train_dataset[-25:])
print_images(generator.predict(np.random.normal(0, 1, size=[25, dim_random_vector])))

plt.plot(errors_generator, label='generator')
plt.plot(errors_discrim, label='discriminator')
plt.legend()

plt.figure()
plt.plot(perc_discrim)

O modelo parece bem efetivo em aprender como gerar alguns poucos números. Será que o modelo nessas mesmas configurações seria capaz de gerar todos os números de 0 a 9?

Desenvolva uma função que separa a base de dados em subconjuntos que só possuem exemplos com 1 tipo de número, e treine uma GAN em cada um desses subconjuntos. Todos são capazes de gerar números realísticos? Ou só algum deles?

Vamos explorar agora um modelo gerador que também utiliza convoluções. Para isso utilize a classe [UpSampling2D](https://keras.io/api/layers/reshaping_layers/up_sampling2d/) para aumentar as dimensões da saída do modelo.

In [ ]:
def gan_generator_model(dim_random):
    #Exemplo abaixo, sinta-se livre para alterá-lo
    model = Sequential()
    model.add(Dense(128*7*7, input_dim=dim_random, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, kernel_size=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(28*28, activation='tanh'))
    model.add(Reshape((28,28,1)))
    return model

generator = gan_generator_model(dim_random_vector)
generator.summary()
discriminator = gan_discriminator_model()
opt = Adam(lr=0.0002, beta_1=0.5)
discriminator.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
discriminator.summary()
gan = define_gan(generator, discriminator)
gan.summary()

Vamos treinar esse novo modelo

In [ ]:
train_gan(generator, discriminator, gan)

In [ ]:
print_images(train_dataset[-25:])
print_images(generator.predict(np.random.normal(0, 1, size=[25, dim_random_vector])))

plt.plot(errors_generator, label='generator')
plt.plot(errors_discrim, label='discriminator')
plt.legend()

plt.figure()
plt.plot(perc_discrim)

Faça o mesmo que você fez para o primeiro modelo e treine um modelo separado para cada dígito possível. Qual modelo é mais efetivo para cada dígito?